In [ ]:
import torch
from utils import *
from dataset import ImageDataset
from models.swin_unet import SwinUnet
from dataset import ImageDataset
import os
from glob import glob

from tqdm import tqdm
from utils import *
from consts import *
import cv2
import numpy as np
from datetime import datetime
from PIL.Image import fromarray

In [ ]:
checkpoint = torch.load("../checkpoints/omgggg.pt", map_location=torch.device('cpu'))
model = SwinUnet(model_type="base").to("cpu")
model.load_state_dict(checkpoint["model_state_dict"])

In [ ]:
device = "cpu"
test_path = "../data/test"


print("Predicting on test set...")
test_path = os.path.join(test_path, "images")
test_filenames = glob(test_path + "/*.png")
test_images = load_all_from_path(test_path)
batch_size = test_images.shape[0]
size = test_images.shape[1:3]
# we also need to resize the test images. This might not be the best ideas depending on their spatial resolution.
print("Resizing test images...")
test_images = np.stack([img for img in test_images], 0)
test_images = test_images[:, :, :, :3]
test_images = np_to_tensor(np.moveaxis(test_images, -1, 1), device)


In [ ]:
%matplotlib inline

print("Making predictions...")
import torchvision.transforms.functional as TF

with torch.no_grad():
    # checkpoint = torch.load(best_weights_path)
    # model.load_state_dict(checkpoint["model_state_dict"])
    # print(f"Loaded best model weights ({best_weights_path})")

    test_pred = []
    CROP_SIZE = 200
    RESIZE_SIZE = 208
    for image in tqdm(test_images):
        # 1. On fait les crops: on divise l'image en 4 parties qui ne s'overlapent pas
        cropped_image = [
            image[:, 0:CROP_SIZE, 0:CROP_SIZE],
            image[:, CROP_SIZE : 2 * CROP_SIZE, 0:CROP_SIZE],
            image[:, 0:CROP_SIZE, CROP_SIZE : 2 * CROP_SIZE],
            image[:, CROP_SIZE : 2 * CROP_SIZE, CROP_SIZE : 2 * CROP_SIZE],
        ]

        # pred_cropped = [
        #     model(cv2.resize(c_img, dsize=(RESIZE_SIZE, RESIZE_SIZE)))
        #     .detach()
        #     .cpu()
        #     .numpy()
        #     for c_img in cropped_image
        # ]
        pred_cropped = []
        for c_img in cropped_image:
            r = TF.resize(c_img, size=(RESIZE_SIZE, RESIZE_SIZE))
            r = r[None, :]
            res = model(r).detach().cpu()
            print(res.shape)
            print(torch.max(res))
            pred_cropped.append(res)

        full_pred = np.zeros((3, 400, 400))

        full_pred[:, 0:CROP_SIZE, 0:CROP_SIZE] = TF.resize(
            pred_cropped[0], size=(CROP_SIZE, CROP_SIZE)
        )
        full_pred[:, CROP_SIZE : 2 * CROP_SIZE, 0:CROP_SIZE] = TF.resize(
            pred_cropped[1], size=(CROP_SIZE, CROP_SIZE)
        )
        full_pred[:, 0:CROP_SIZE, CROP_SIZE : 2 * CROP_SIZE] = TF.resize(
            pred_cropped[2], size=(CROP_SIZE, CROP_SIZE)
        )
        full_pred[
            :, CROP_SIZE : 2 * CROP_SIZE, CROP_SIZE : 2 * CROP_SIZE
        ] = TF.resize(pred_cropped[3], size=(CROP_SIZE, CROP_SIZE))

        test_pred.append(full_pred)

    # test_pred = [model(t).detach().cpu().numpy()
    #              for t in tqdm(test_images.unsqueeze(1))]



In [ ]:
test_pred_copy = test_pred.copy()

In [ ]:
np.max(test_pred[0])

In [ ]:

with torch.no_grad():
    test_pred_c = np.concatenate(test_pred_copy, 0)
    test_pred_c = np.moveaxis(test_pred, 1, -1)  # CHW to HWC
    test_pred_c = np.stack([img for img in test_pred], 0)  # resize to original shape
    # Now compute labels
    test_pred_c = test_pred_c.reshape(
        (-1, size[0] // PATCH_SIZE, PATCH_SIZE, size[0] // PATCH_SIZE, PATCH_SIZE)
    )
    test_pred_c = np.moveaxis(test_pred_c, 2, 3)
    test_pred_c = np.round(np.mean(test_pred_c, (-1, -2)) > CUTOFF)
    print(f"Test predictions shape: {test_pred_c.shape}")
    now = datetime.now()
    t = now.strftime("%Y-%m-%d_%H-%M-%S")
    os.makedirs("submissions", exist_ok=True)
    create_submission(
        test_pred_c,
        test_filenames,
        submission_filename=f"../submissions/swin_unet_submission_{t}.csv",
    )
    print(f"Created submission!")